# Profitable App Profiles for the App Store and Google Play Markets

My aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. 

In [1]:
from csv import reader
open_android = open('googleplaystore.csv')
read_android = reader(open_android)
android = list(read_android)
android_header = android[0]
android = android[1:]

open_ios = open('AppleStore.csv')
read_ios = reader(open_ios)
ios = list(read_ios)
ios_header = ios[0]
ios = ios[1:]

print('PlayMarket: \n', android_header)
print('AppStore: \n', ios_header)

PlayMarket: 
 ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
AppStore: 
 ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


#### Deleting wrong data

In [6]:
def check_correctness(dataset, header):
    for row in dataset:
        if len(row) != len(header):
            print(row)
            print(dataset.index(row))

check_correctness(android, android_header)
check_correctness(ios, ios_header)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10472
['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


#### Checking for duplicates

In [7]:
def duplicates_check(dataset, header):
    duplicate_apps = []
    unique_apps = []
    for app in dataset:
        name = app[0]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    name = header[0]
    if name == 'App':
        data = 'Play Market'
    else:
        data = 'AppStore'
    print("Number of duplicate apps in ", data,": ", len(duplicate_apps))
    print("\n")
    print("Example of duplicate apps:", duplicate_apps[:10])
    print("\n")
duplicates_check(android, android_header)
duplicates_check(ios, ios_header)
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

Number of duplicate apps in  Play Market :  1181


Example of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


Number of duplicate apps in  AppStore :  0


Example of duplicate apps: []


['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies w

#### Removing duplicates 
As only **Play Market ** dataset contains duplicates we will use following criteria: The higher the number of reviews, the more recent the data should be.

In [8]:
for app in android:
    if app[3] == '3.0M':
        print(app)

In [9]:
reviews_max = {} # dictionary to store app name and maximum number of reviews

for app in android: # adding values into dictionary
    name = app[0]
    reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < reviews:
        reviews_max[name] = reviews
    else:
        reviews_max[name] = reviews

print('Expected length: ', len(android) - 1181)
print('Actual length: ', len(reviews_max))

Expected length:  9659
Actual length:  9659


We have checked and saved maximum review with app name, and now let's remove duplicates

In [13]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    reviews = float(app[3])
    if reviews_max[name] == reviews and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
print(len(android_clean))

9659


We have 9659 rows in _android_ _clean_ as expected

#### Removing non-english apps

In [23]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

True
True


In [32]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)

for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)  
        
print(len(android_english))
print(len(ios_english))

9614
6183


In [33]:
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)
for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)

print(len(android_final))
print(len(ios_final))

8864
3222


#### Most common apps by genre
let's find out successful genres in both markets
1. _freq table_ function to generate frequency tables that show percentages
2. _display table_ function that we can use to display the percentages in a descending order

In [36]:
def freq_table(dataset, index):
    table = {}
    total = 0
    for row in dataset:
        total += 1
        genre = row[index]
        if genre in table:
            table[genre] += 1
        else:
            table[genre] = 1
    
    final_table = {}
    for key in table:
        percentage =  (table[key]/total) * 100
        final_table[key] = percentage
    
    return final_table

def display_table(dataset, index):
    table = freq_table(dataset,index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
    

In [37]:
display_table(ios_final, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [38]:
display_table(android_final, 1)

FAMILY : 19.223826714801444
GAME : 9.510379061371841
TOOLS : 8.461191335740072
BUSINESS : 4.580324909747293
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.5424187725631766
SPORTS : 3.4183212996389893
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2490974729241873
HEALTH_AND_FITNESS : 3.068592057761733
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.782490974729242
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.128158844765343
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
ENTERTAINMENT : 0.8799638989169676
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 0

In [40]:
display_table(android_final, -4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.580324909747293
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.5424187725631766
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2490974729241873
Action : 3.1024368231046933
Health & Fitness : 3.068592057761733
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.861462093862816
Video Players & Editors : 1.782490974729242
Casual : 1.7486462093862816
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.925090252707581

Practical apps seem to have a better representation on Google Play compared to App Store.

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

#### Most popular apps by genre in AppStore

In [41]:
genres_ios = freq_table(ios_final, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:
            ratings = float(app[5])
            total += ratings
            len_genre += 1
    avg_ratings = total / len_genre
    print(genre, ':', avg_ratings)

Medical : 612.0
Navigation : 86090.33333333333
Games : 22788.6696905016
Weather : 52279.892857142855
Health & Fitness : 23298.015384615384
Catalogs : 4004.0
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Entertainment : 14029.830708661417
Lifestyle : 16485.764705882353
Sports : 23008.898550724636
Music : 57326.530303030304
Shopping : 26919.690476190477
Travel : 28243.8
Finance : 31467.944444444445
Productivity : 21028.410714285714
Education : 7003.983050847458
Book : 39758.5
Business : 7491.117647058823
Utilities : 18684.456790123455
News : 21248.023255813954
Photo & Video : 28441.54375
Food & Drink : 33333.92307692308



On average, navigation apps have the highest number of user reviews

In [42]:
for app in ios_final:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


As we can see, Navigation genre is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together worldwide

#### Most popular apps by genre in Google Play

In [44]:
category_android = freq_table(android_final, 1)

for category in category_android:
    total = 0
    len_category = 0
    for app in android_final:
        category_app = app[1]
        if category_app == category:
            installs = app[5]
            installs = installs.replace(',', '')
            installs = installs.replace('+', '')
            total += float(installs)
            len_category += 1
    avg_installs = total/len_category
    print(category, ': ', avg_installs)

SOCIAL :  23253652.127118643
SPORTS :  4274688.722772277
TRAVEL_AND_LOCAL :  13984077.710144928
VIDEO_PLAYERS :  24790074.17721519
LIBRARIES_AND_DEMO :  638503.734939759
COMICS :  817657.2727272727
FAMILY :  5180161.789906103
SHOPPING :  7036877.311557789
LIFESTYLE :  1437816.2687861272
EVENTS :  253542.22222222222
NEWS_AND_MAGAZINES :  9549178.467741935
HOUSE_AND_HOME :  1331540.5616438356
ART_AND_DESIGN :  1986335.0877192982
BUSINESS :  1704192.3399014778
PRODUCTIVITY :  16772838.591304347
PERSONALIZATION :  5201482.6122448975
MAPS_AND_NAVIGATION :  4056941.7741935486
ENTERTAINMENT :  9146923.076923076
DATING :  854028.8303030303
PHOTOGRAPHY :  17840110.40229885
COMMUNICATION :  38326063.197916664
BEAUTY :  513151.88679245283
GAME :  12914435.883748516
MEDICAL :  123064.7898089172
PARENTING :  542603.6206896552
BOOKS_AND_REFERENCE :  8767811.894736841
WEATHER :  5074486.197183099
HEALTH_AND_FITNESS :  4167457.3602941176
FINANCE :  1387692.475609756
FOOD_AND_DRINK :  1924897.736363636

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [45]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Messenger – Text and Video Chat for Free : 1,000,000,000+
Gmail : 1,000,000,000+
imo beta free calls and text : 100,000,000+
imo free video calls and chat : 500,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
WhatsApp Messenger : 1,000,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Hangouts : 1,000,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

In [47]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

So, building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

Conclusions
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.